
Copyright(c) 2025 NVIDIA Corporation. All rights reserved.

NVIDIA Corporation and its licensors retain all intellectual property
and proprietary rights in and to this software, related documentation
and any modifications thereto.Any use, reproduction, disclosure or
distribution of this software and related documentation without an express
license agreement from NVIDIA Corporation is strictly prohibited.

# Add parameters to the config.yaml for the reference workflow

This notebook illustrates how you can move your frequently used parameters to the config file for the reference workflow. In the example here, we will move the voice_id parameter used by the default TTS provider (elevenLabs) as a config paramter. 

### Ensure that the Pre-requisites are met

Please run through the steps noted in the setup prequisites notebook to ensure that you have the necessary setup in place.

Check the status of the application again by running the following command. 

Note: You will need to select the python environment to run the notebook for the first time. 

In [ ]:
!kubectl get po -n app

### Download the ACE Controller Configuration
* Ensure that the downloaded ACE Controller configuration is in sync. You can click on the download button again to download the latest version of the configuration. 

![ACE Configurator](pictures/ace_configurator_sync.png)

### Edit the `config.yaml` file
1. Navigate to the following path in your file system:
   ```
   ace-controller-ace-controller-deployment -> assets -> app-storage-volume -> configs -> config.yaml
   ```
2. Open the `config.yaml` file.
3. Add the following the following section anywhere, as shown by the sample snippet

In [ ]:
Pipeline:
    # Only one of the following LLM service configurations will be active based on this setting:
    # - "NvidiaLLMService" - Uses the NvidiaLLMService configuration
    # - "NvidiaRAGService" - Uses the NvidiaRAGService configuration
    # - "OpenAILLMService" - Uses the OpenAILLMService configuration
    llm_processor: "OpenAILLMService" # OR NvidiaLLMService OR NvidiaRAGService OR OpenAILLMService
    filler:
        - "Let me think"
        - "Hmmm"
    time_delay: 2.0

TTSProcessor:
    voice_id: "XrExE9yKIg1WjnnlVkGX"

UserPresenceProcesssor:
    welcome_message: "Hello"
    farewell_message: "Bye"

Update the `config.py` file to ensure that the additional configuration is read successfully. The file would be located in the downloaded folder. 

   ```
   ace-controller-ace-controller-deployment -> assets -> app-storage-volume -> src -> config.py
   ```

In [ ]:

# Define the pydantic class for your processor in the file. 
class TTSProcessor(BaseModel):
    voice_id: StrictStr = "XrExE9yKIg1WjnnlVkGX"

# Root model for the pipeline configuration
class Config(BaseModel):
    Pipeline: Pipeline
    TTSProcessor: TTSProcessor  #<------- Add your processor here

Navigate to the `bot.py` in the same folder and now update the code to read the `voice_id` setting from the config.yaml file. 

In [ ]:
tts = ElevenLabsTTSServiceWithEndOfSpeech(
        api_key=os.getenv("ELEVENLABS_API_KEY"),
        voice_id=config.TTSProcessor.voice_id,
        sample_rate=16000,
        model = "eleven_flash_v2_5",
    )


### Upload the Changes
Save your local changes in VSCode. 
In the ACE Configurator view in VSCode, click the **Upload** button to upload the modified `config.yaml` file.

![Avatar Controller Upload Config](pictures/upload_config.png)

Ensure that the changes are finished updating. This might take a few seconds.

Once the changes are successfully updated. the ACE Configurator view will show a green `Synced` symbol again for the app-storage-volume. 

![Avatar Controller Upload Config](pictures/ace-configurator-synced.png)

### Refresh the Browser UI
1. Open the browser UI where the digital human is hosted. (your current URL:30111)
2. Refresh the page.
3. Start a conversation with the digital human, and you should see it is using the voice_id configured from the config.yaml of your reference workflow.

### Conclusion
You have successfully reconfigured the ACE digital human and tested the changes. 🎉

This particular process of moving settings to the config.yaml can be extended to a multitude of parameters used by the `bot.py` file. The default reference workflow uses limited number of parameters from the config file to maintain simplicity. 